In [15]:
!pip install torch
!pip install bertopic sentence-transformers umap-learn hdbscan

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
import json

In [ ]:
data = []
with open("dataset.jsonl", "r") as f:
    data = [json.loads(line) for line in f]
docs = [f"code:{item['code']} query:{item['query']}" for item in data[:1000]]
print(len(data))
print(len(docs))

237200
1000


In [32]:
from typing import List


model = BERTopic(
    embedding_model="all-MiniLM-L6-v2",  # Efficient embedding model for quick computation
    umap_model=UMAP(
        n_neighbors=10,  # Lower neighbors for tighter clusters
        n_components=5,  # Dimensionality reduction
        min_dist=0.1,  # Slight separation between clusters
        metric="cosine",  # Cosine similarity for text data
    ),
    hdbscan_model=HDBSCAN(
        min_cluster_size=60,  # Increase to reduce topic count
        min_samples=15,  # Fewer samples to prevent fragmentation
        metric="euclidean",  # Works well with UMAP output
        cluster_selection_method="eom",  # Keeps distinct clusters
    ),  # Target topic count, reduces smaller ones
    top_n_words=10,  # Top words per topic
)
topics, probs = model.fit_transform(docs)
model.visualize_barchart()

In [ ]:
import json
from collections import defaultdict

topic_data_map = defaultdict(list)

for i, topic_assignment in enumerate(topics):
    document_data = docs[i]
    document_probs = probs[i]

    topic_data_map[topic_assignment].append((document_data, document_probs))


json_output = json.dumps(topic_data_map, indent=4)

with open("cluster.json", "w") as f:
    f.write(json_output)

   Topic  Count                     Name  \
0     -1     34     -1_pid_the_if_return   
1      0    544         0_self_the_if_in   
2      1    422  1_self_the_in_tenant_id   

                                      Representation  \
0  [pid, the, if, return, searchportion, for, non...   
1  [self, the, if, in, return, to, for, code, of,...   
2  [self, the, in, tenant_id, get, if, to, for, l...   

                                 Representative_Docs  
0  [code:def scanProcessForOpenFile(pid, searchPo...  
1  [code:def exists(self, value=None):\n        "...  
2  [code:def _check_create_fw(self, tenant_id, dr...  
